#### Import Library

###### 차선 검출에 필요한 Library 들을 현재 파일에 호출한다.

In [2]:
# -*- coding: utf-8 -*-
import cv2
import math
import numpy as np

#### 함수 정의

##### Step 1
###### 이 함수들을 통해 차선을 검출하는 과정에서 특정 영역을 설정 및 불필요한 부분 제거가 가능하다.

In [3]:
# HSV 조절 시, 사용할 빈 Callback 함수 -> 값의 변화를 반영시킬 때 사용함
def nothing(x):
    pass

# 입력 이미지에서 관심 영역(ROI, Region of Interest)을 설정하는 함수
# 다각형 형태의 관심 영역을 마스킹하여 다른 부분을 가림
def region_of_interest(img, roi):
    vertices = np.array([roi], np.int32)
    mask = np.zeros_like(img)
    match_mask_color = 255
    cv2.fillPoly(mask, vertices, match_mask_color)
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

##### Step 2

###### 이미지 엣지 검출 및 차선 찾기 함수 정의

In [4]:
# Using Canny to find edges
def find_edges(img, h_min=0, h_max=0, s_min=0, s_max=0, v_min=0, v_max=0):
    height = img.shape[0]
    width = img.shape[1]
    
    roi = [
        (0, height),
        (0, height / 2),
        (width, height / 2),
        (width, height)
    ]

    # Blur
    img = cv2.GaussianBlur(img, (5, 5), 0)

    # Change image channel to HSV
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    low_val = (h_min, s_min, v_min)
    high_val = (h_max, s_max, v_max)

    # Threshold
    mask = cv2.inRange(hsv, low_val, high_val)

    # Crop the image
    mask = region_of_interest(mask, roi)

    # find edges
    edges = cv2.Canny(mask, 75, 150)
    return edges


##### Step 3

###### 차선 검출 및 차선 그리기 함수 정의

In [5]:
# After Canny & Hough Transform, find a line
def find_lines(image, lines):
    width = image.shape[1]

    # init list
    lane_lines = []
    left_fit = []
    right_fit = []
    
    boundary = 1 / float(3)
    left_region_boundary = width * (1 - float(boundary))
    right_region_boundary = width * float(boundary)

    for line in lines:
        for x1, y1, x2, y2 in line:
            if x1 == x2:
                continue
            fit = np.polyfit((x1, x2), (y1, y2), 1)
            slope = fit[0]
            intercept = fit[1]

            if slope < 0:
                if x1 < left_region_boundary and x2 < left_region_boundary:
                    left_fit.append((slope, intercept))
            else:
                if x1 > right_region_boundary and x2 > right_region_boundary:
                    right_fit.append((slope, intercept))

    if len(left_fit) > 0:
        left_fir_average = np.average(left_fit, axis=0)
        lane_lines.append(make_points(image, left_fir_average))
    if len(right_fit) > 0:
        right_fit_average = np.average(right_fit, axis=0)
        lane_lines.append(make_points(image, right_fit_average))

    return lane_lines

# Draw detected lanes
def draw_line(image, lane_lines, current_theta):
    height = image.shape[0]
    width = image.shape[1]

    if len(lane_lines) == 2:
        _, _, left_x2, _ = lane_lines[0][0]
        _, _, right_x2, _ = lane_lines[1][0]
        mid = int(width / float(2))

        base_x = int(left_x2 + right_x2) / 2 - float(mid)
        base_y = int(height / float(2))
        image, current_theta = display_base_line(image, current_theta, 2, base_x, base_y)
    elif len(lane_lines) == 1:
        x1, _, x2, _ = lane_lines[0][0]
        base_x = x2 - x1
        base_y = int(height / float(2))
        image, current_theta = display_base_line(image, current_theta, 1, base_x, base_y)
    else:
        base_x = width / float(2)
        base_y = int(height / float(2))
        image, current_theta = display_base_line(image, 0, 0, base_x, base_y)

    for line in lane_lines:
        for x1, y1, x2, y2 in line:
            cv2.line(image, (x1, y1), (x2, y2), (0, 255, 0), 7)

    return image, current_theta

# Create points for lane lines
def make_points(frame, line):
    height, width, _ = frame.shape
    slope, intercept = line
    y1 = height  # bottom of the frame
    y2 = int(y1 / float(2))  # middle of the frame

    if slope == 0:
        slope = 1
    x1 = max(-width, min(2 * width, int((y1 - intercept) / float(slope))))
    x2 = max(-width, min(2 * width, int((y2 - intercept) / float(slope))))

    return [[x1, y1, x2, y2]]

##### Step 4

###### 차선 방향과 각도 안정화 관련 함수 정의

In [6]:
# Display base line to determine lane curvature
def display_base_line(image, current_theta, line_num, base_x, base_y):
    height = image.shape[0]
    width = image.shape[1]
    
    angle_to_mid_radian = math.atan(base_x / float(base_y))
    angle_to_mid_deg = int(angle_to_mid_radian * 180 / float(3.14))
    angle_to_mid_deg = stabilize_angle(current_theta, angle_to_mid_deg, number_of_line=line_num)

    theta = angle_to_mid_deg + 90
    steering_angle_radian = theta / float(180) * float(3.14)

    x1 = int(width / float(2))
    y1 = height
    if steering_angle_radian == 0:
        x2 = int(width / float(2))
    else:
        x2 = int(x1 - height * 2 / 3 / float(math.tan(steering_angle_radian)))

    y2 = int(height / float(2))

    result = np.zeros_like(image)
    cv2.line(result, (x1, y1), (x2, y2), (0, 255, 255), 4)
    result = cv2.addWeighted(image, 0.8, result, 1, 1)

    return result, angle_to_mid_deg

# Stabilize the detected lane angle
def stabilize_angle(curr_theta, new_theta, number_of_line, max_twoLine_angle_offset=1, max_signleLine_angle_offset=1):
    if number_of_line == 2:
        max_angle_offset = max_twoLine_angle_offset
    elif number_of_line == 1:
        max_angle_offset = max_signleLine_angle_offset
    else:
        return 0

    angle_offset = new_theta - curr_theta
    if abs(angle_offset) > max_angle_offset:
        fixed_angle = curr_theta + max_angle_offset * (angle_offset / abs(angle_offset))
    else:
        fixed_angle = new_theta

    if fixed_angle > 20:
        fixed_angle = 20
    elif fixed_angle < -20:
        fixed_angle = -20

    return fixed_angle

##### Step 5

###### 주된 차선 검출 로직 함수 정의

In [7]:
# Main lane detection function
def line_detect(image, current_theta=0, h_min=0, h_max=179, s_min=0, s_max=255, v_min=190, v_max=255):
    height = image.shape[0]
    width = image.shape[1]

    edges = find_edges(image, h_min, h_max, s_min, s_max, v_min, v_max)
    cv2.imshow('Canny', edges)

    lines = cv2.HoughLinesP(edges, 1, float(3.14) / float(180), 50, maxLineGap=100)
    
    if lines is not None:
        lane_lines = find_lines(image, lines=lines)
        if lane_lines is not None:
            image, current_theta = draw_line(image, lane_lines=lane_lines, current_theta=current_theta)

    if current_theta > 0:
        direction = 'Right'
    elif current_theta < 0:
        direction = 'Left'
    else:
        direction = 'Straight'

    image = cv2.putText(image, direction + ' ' + str(current_theta),
                        (int(height - 200), int(width / float(2))),
                        cv2.FONT_HERSHEY_PLAIN,
                        3, (255, 255, 0))

    return image, current_theta, direction

##### Step 6

###### Video 파일 또는 웹캠을 통한 영상에서 차선을 검출하기 위한 함수 정의

In [8]:
# HSV 슬라이더를 통해 값을 조절할 수 있는 기능 추가
def process_video_with_hsv_control():
    cap = cv2.VideoCapture(0)  # 웹캠 사용
    if not cap.isOpened():
        print("웹캠을 열 수 없습니다. 웹캠이 제대로 연결되어 있는지 확인해주세요.")
        return

    current_theta = 0

    # HSV 컨트롤 창 생성
    cv2.namedWindow('Control')
    cv2.createTrackbar('h_min', 'Control', 0, 179, nothing)
    cv2.createTrackbar('h_max', 'Control', 0, 179, nothing)
    cv2.createTrackbar('s_min', 'Control', 0, 255, nothing)
    cv2.createTrackbar('s_max', 'Control', 0, 255, nothing)
    cv2.createTrackbar('v_min', 'Control', 0, 255, nothing)
    cv2.createTrackbar('v_max', 'Control', 0, 255, nothing)

    # 초기 HSV 슬라이더 값 설정
    cv2.setTrackbarPos('h_min', 'Control', 0)
    cv2.setTrackbarPos('h_max', 'Control', 179)
    cv2.setTrackbarPos('s_min', 'Control', 0)
    cv2.setTrackbarPos('s_max', 'Control', 255)
    cv2.setTrackbarPos('v_min', 'Control', 190)
    cv2.setTrackbarPos('v_max', 'Control', 255)

    while cap.isOpened():
        ret, frame = cap.read()
        if ret:
            frame = cv2.resize(frame, dsize=(640, 480))

            # HSV 슬라이더 값 읽기
            h_min = cv2.getTrackbarPos('h_min', 'Control')
            h_max = cv2.getTrackbarPos('h_max', 'Control')
            s_min = cv2.getTrackbarPos('s_min', 'Control')
            s_max = cv2.getTrackbarPos('s_max', 'Control')
            v_min = cv2.getTrackbarPos('v_min', 'Control')
            v_max = cv2.getTrackbarPos('v_max', 'Control')

            # 실시간 차선 검출 수행
            img, current_theta, direction = line_detect(frame, current_theta, h_min, h_max, s_min, s_max, v_min, v_max)
            cv2.imshow('result', img)

            if cv2.waitKey(50) & 0xFF == ord('q'):
                break
        else:
            break

    cap.release()
    cv2.destroyAllWindows()

#### 코드 전체 실행

In [9]:
process_video_with_hsv_control()

error: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window_w32.cpp:2561: error: (-27:Null pointer) NULL window: 'Control' in function 'cvGetTrackbarPos'


: 